# Bagging Forests Category - Kaggle Playground Series S5E8

**Category**: Bagging & Random Forests  
**Sub-models**: RandomForestClassifier, ExtraTreesClassifier  
**Split Strategy**: 70/30 stratified split  
**Cross-Validation**: 5-fold StratifiedKFold  
**Random Seed**: 42  
**Artifact Paths**: outputs/bagging_forests/  

This notebook compares ensemble tree-based models using bagging and random feature selection.

In [1]:
# Bootstrap installation and imports
%pip install numpy pandas scikit-learn matplotlib --quiet

import os, json, random, pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import (
    roc_auc_score, average_precision_score, f1_score, accuracy_score,
    precision_score, recall_score, log_loss, roc_curve, precision_recall_curve,
    confusion_matrix
)
from sklearn.calibration import calibration_curve
import warnings
warnings.filterwarnings('ignore')

# Set random seeds
os.makedirs('outputs', exist_ok=True)
np.random.seed(42)
random.seed(42)

print("Bagging Forests Category - Setup Complete")

Note: you may need to restart the kernel to use updated packages.
Bagging Forests Category - Setup Complete
Bagging Forests Category - Setup Complete


In [2]:
# Load and prepare data
train_df = pd.read_csv('../playground-series-s5e8/train.csv')
test_df = pd.read_csv('../playground-series-s5e8/test.csv')

feature_cols = [col for col in train_df.columns if col not in ['id', 'y']]
X = train_df[feature_cols]
y = train_df['y']

X_train_pool, X_test_holdout, y_train_pool, y_test_holdout = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
print(f"Data loaded: {X_train_pool.shape} train, {X_test_holdout.shape} test")
print(f"Features: {len(feature_cols)}")
print(f"Target distribution: {np.bincount(y)}")

Data loaded: (525000, 16) train, (225000, 16) test
Features: 16
Target distribution: [659512  90488]


In [3]:
# Explore data structure first
train_df = pd.read_csv('../playground-series-s5e8/train.csv')
test_df = pd.read_csv('../playground-series-s5e8/test.csv')

print("Train data shape:", train_df.shape)
print("Train columns:", list(train_df.columns))
print("\nTest data shape:", test_df.shape)  
print("Test columns:", list(test_df.columns))

print("\nFirst few rows of train data:")
print(train_df.head())

Train data shape: (750000, 18)
Train columns: ['id', 'age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']

Test data shape: (250000, 17)
Test columns: ['id', 'age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome']

First few rows of train data:
   id  age          job  marital  education default  balance housing loan  \
0   0   42   technician  married  secondary      no        7      no   no   
1   1   38  blue-collar  married  secondary      no      514      no   no   
2   2   36  blue-collar  married  secondary      no      602     yes   no   
3   3   27      student   single  secondary      no       34     yes   no   
4   4   26   technician  married  secondary      no      889     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome  

In [4]:
# Define Random Forest and Extra Trees models
models_config = {
    'RandomForest_balanced': {
        'estimator': RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1),
        'param_grid': {
            'classifier__n_estimators': [300, 800],
            'classifier__max_depth': [None, 12, 20]
        }
    },
    'RandomForest_default': {
        'estimator': RandomForestClassifier(random_state=42, n_jobs=-1),
        'param_grid': {
            'classifier__n_estimators': [300, 800],
            'classifier__max_depth': [None, 12, 20]
        }
    },
    'ExtraTrees_balanced': {
        'estimator': ExtraTreesClassifier(random_state=42, class_weight='balanced', n_jobs=-1),
        'param_grid': {
            'classifier__n_estimators': [300, 800],
            'classifier__max_depth': [None, 12, 20]
        }
    },
    'ExtraTrees_default': {
        'estimator': ExtraTreesClassifier(random_state=42, n_jobs=-1),
        'param_grid': {
            'classifier__n_estimators': [300, 800],
            'classifier__max_depth': [None, 12, 20]
        }
    }
}

print(f"Configured {len(models_config)} ensemble tree variants:")
for name in models_config.keys():
    print(f"  - {name}")

Configured 4 ensemble tree variants:
  - RandomForest_balanced
  - RandomForest_default
  - ExtraTrees_balanced
  - ExtraTrees_default


In [ ]:
# Helper functions
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

def create_pipeline(estimator):
    """Create preprocessing pipeline - handle both categorical and numerical features"""
    # Identify categorical and numerical columns
    categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = X.select_dtypes(include=['number']).columns.tolist()
    
    print(f"    Pipeline setup:")
    print(f"      Categorical columns ({len(categorical_cols)}): {categorical_cols}")
    print(f"      Numerical columns ({len(numerical_cols)}): {numerical_cols}")
    print(f"      Preprocessing: SimpleImputer(median) + OneHotEncoder(drop_first)")
    
    # Create preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', SimpleImputer(strategy='median'), numerical_cols),
            ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), categorical_cols)
        ]
    )
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', estimator)
    ])
    
    print(f"      Final pipeline: {' -> '.join(pipeline.named_steps.keys())}")
    return pipeline

def get_probabilities(estimator, X):
    """Extract positive class probabilities from trained estimator"""
    probs = estimator.predict_proba(X)[:, 1]
    print(f"      Generated probabilities: min={probs.min():.4f}, max={probs.max():.4f}, mean={probs.mean():.4f}")
    return probs

def compute_metrics(y_true, y_prob, threshold=0.5):
    """Compute comprehensive classification metrics"""
    y_pred = (y_prob >= threshold).astype(int)
    
    metrics = {
        'roc_auc': roc_auc_score(y_true, y_prob),
        'average_precision': average_precision_score(y_true, y_prob),
        'f1': f1_score(y_true, y_pred),
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'logloss': log_loss(y_true, y_prob)
    }
    
    return metrics

def find_best_threshold(y_true, y_prob):
    """Find optimal threshold using Youden's J statistic"""
    fpr, tpr, thresholds = roc_curve(y_true, y_prob)
    j_scores = tpr - fpr
    best_idx = np.argmax(j_scores)
    best_threshold = thresholds[best_idx]
    best_j_score = j_scores[best_idx]
    
    print(f"      Optimal threshold: {best_threshold:.4f} (J-score: {best_j_score:.4f})")
    return best_threshold

print("✅ Helper functions defined with detailed logging")
print("\nData preprocessing summary:")
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=['number']).columns.tolist()
print(f"  Categorical features ({len(categorical_cols)}): {categorical_cols}")
print(f"  Numerical features ({len(numerical_cols)}): {numerical_cols}")
print(f"  Total original features: {len(categorical_cols) + len(numerical_cols)}")
print(f"  Expected features after OneHot: {len(numerical_cols)} + (encoded categoricals)")

# Show unique values for categorical columns
print(f"\nCategorical feature cardinality:")
for col in categorical_cols[:5]:  # Show first 5 to avoid too much output
    n_unique = X[col].nunique()
    print(f"  {col}: {n_unique} unique values")
if len(categorical_cols) > 5:
    print(f"  ... and {len(categorical_cols) - 5} more categorical features")

Helper functions defined
Categorical columns: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
Numerical columns: ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']


In [ ]:
# Main evaluation loop
results = {}

print("Starting evaluation of all bagging forest models...")
print(f"Total models to evaluate: {len(models_config)}")
print(f"Cross-validation strategy: {cv.n_splits}-fold StratifiedKFold")
print("=" * 60)

for model_idx, (model_name, config) in enumerate(models_config.items(), 1):
    print(f"\n[{model_idx}/{len(models_config)}] Evaluating: {model_name}")
    print(f"Estimator: {type(config['estimator']).__name__}")
    print(f"Class weight: {getattr(config['estimator'], 'class_weight', 'default')}")
    
    model_dir = f"../outputs/bagging_forests/{model_name}"
    print(f"Creating output directories: {model_dir}")
    os.makedirs(f"{model_dir}/logs", exist_ok=True)
    os.makedirs(f"{model_dir}/models", exist_ok=True)
    os.makedirs(f"{model_dir}/figures", exist_ok=True)
    
    print("Creating preprocessing pipeline...")
    pipeline = create_pipeline(config['estimator'])
    print(f"Pipeline steps: {list(pipeline.named_steps.keys())}")
    
    print("Starting grid search...")
    print(f"Parameter grid: {config['param_grid']}")
    grid_search = GridSearchCV(
        pipeline, config['param_grid'], cv=cv, 
        scoring='roc_auc', n_jobs=1, verbose=1  # n_jobs=1 to avoid nested parallelism
    )
    
    print("Fitting grid search on training data...")
    grid_search.fit(X_train_pool, y_train_pool)
    best_pipeline = grid_search.best_estimator_
    
    print(f"Grid search completed!")
    print(f"Best params: {grid_search.best_params_}")
    print(f"Best CV AUC: {grid_search.best_score_:.4f}")
    
    # CV analysis
    print("\nStarting detailed cross-validation analysis...")
    cv_metrics = []
    cv_roc_curves = []
    cv_pr_curves = []
    cv_thresholds = []
    
    for fold_idx, (train_idx, val_idx) in enumerate(cv.split(X_train_pool, y_train_pool)):
        print(f"  Processing fold {fold_idx + 1}/{cv.n_splits}...")
        
        X_fold_train = X_train_pool.iloc[train_idx]
        X_fold_val = X_train_pool.iloc[val_idx]
        y_fold_train = y_train_pool.iloc[train_idx]
        y_fold_val = y_train_pool.iloc[val_idx]
        
        print(f"    Train size: {len(X_fold_train)}, Val size: {len(X_fold_val)}")
        print(f"    Target distribution - Train: {np.bincount(y_fold_train)}, Val: {np.bincount(y_fold_val)}")
        
        fold_pipeline = create_pipeline(config['estimator'])
        fold_pipeline.set_params(**grid_search.best_params_)
        
        print(f"    Training fold model...")
        fold_pipeline.fit(X_fold_train, y_fold_train)
        
        print(f"    Making predictions...")
        y_val_prob = get_probabilities(fold_pipeline, X_fold_val)
        best_threshold = find_best_threshold(y_fold_val, y_val_prob)
        cv_thresholds.append(best_threshold)
        
        print(f"    Computing metrics with threshold: {best_threshold:.4f}")
        fold_metrics = compute_metrics(y_fold_val, y_val_prob, best_threshold)
        fold_metrics['fold'] = fold_idx + 1
        fold_metrics['threshold'] = best_threshold
        cv_metrics.append(fold_metrics)
        
        print(f"    Fold {fold_idx + 1} AUC: {fold_metrics['roc_auc']:.4f}")
        
        print(f"    Computing ROC and PR curves...")
        fpr, tpr, _ = roc_curve(y_fold_val, y_val_prob)
        precision, recall, _ = precision_recall_curve(y_fold_val, y_val_prob)
        cv_roc_curves.append((fpr, tpr))
        cv_pr_curves.append((precision, recall))
    
    # Test evaluation
    print("\nEvaluating on test holdout set...")
    mean_threshold = np.mean(cv_thresholds)
    print(f"Using mean threshold from CV: {mean_threshold:.4f}")
    
    y_test_prob = get_probabilities(best_pipeline, X_test_holdout)
    print(f"Generated {len(y_test_prob)} test predictions")
    
    test_metrics = compute_metrics(y_test_holdout, y_test_prob, mean_threshold)
    test_metrics['chosen_threshold'] = mean_threshold
    
    print("Computing confusion matrix...")
    test_metrics['confusion_matrix'] = confusion_matrix(
        y_test_holdout, (y_test_prob >= mean_threshold).astype(int)
    ).tolist()
    
    print(f"Test Results:")
    print(f"  AUC: {test_metrics['roc_auc']:.4f}")
    print(f"  Average Precision: {test_metrics['average_precision']:.4f}")
    print(f"  F1 Score: {test_metrics['f1']:.4f}")
    print(f"  Accuracy: {test_metrics['accuracy']:.4f}")
    
    # Store results
    print("\nStoring results and saving artifacts...")
    results[model_name] = {
        'cv_metrics': cv_metrics,
        'test_metrics': test_metrics,
        'cv_roc_curves': cv_roc_curves,
        'cv_pr_curves': cv_pr_curves,
        'best_params': grid_search.best_params_,
        'model_dir': model_dir
    }
    
    # Save artifacts
    print("Saving CV metrics to CSV...")
    cv_df = pd.DataFrame(cv_metrics)
    cv_df.to_csv(f"{model_dir}/logs/cv_metrics.csv", index=False)
    
    print("Saving test metrics to JSON...")
    with open(f"{model_dir}/logs/test_metrics.json", 'w') as f:
        json.dump(test_metrics, f, indent=2)
    
    print("Saving final model...")
    with open(f"{model_dir}/models/final_model.pkl", 'wb') as f:
        pickle.dump(best_pipeline, f)
    
    print(f"Model {model_name} evaluation completed!")
    print("-" * 60)

print("\n🎉 All forest models evaluated successfully!")
print(f"Results stored for {len(results)} models")


Evaluating: RandomForest_balanced
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best params: {'classifier__max_depth': None, 'classifier__n_estimators': 800}
Best CV AUC: 0.9625
Test AUC: 0.9625

Evaluating: RandomForest_default
Fitting 5 folds for each of 6 candidates, totalling 30 fits


In [ ]:
# Generate figures with forest-specific plots
print("Starting figure generation for all models...")
print(f"Total models to generate figures for: {len(results)}")
print("=" * 60)

for model_idx, (model_name, model_results) in enumerate(results.items(), 1):
    model_dir = model_results['model_dir']
    
    print(f"\n[{model_idx}/{len(results)}] Generating figures for {model_name}...")
    print(f"Output directory: {model_dir}/figures/")
    
    # Load final model
    print("Loading saved model...")
    with open(f"{model_dir}/models/final_model.pkl", 'rb') as f:
        final_model = pickle.load(f)
    
    forest_estimator = final_model.named_steps['classifier']
    print(f"Model type: {type(forest_estimator).__name__}")
    print(f"Number of estimators: {forest_estimator.n_estimators}")
    
    # Get feature names after preprocessing
    print("Extracting feature names after preprocessing...")
    try:
        # Get feature names from the preprocessor
        preprocessor = final_model.named_steps['preprocessor']
        feature_names = preprocessor.get_feature_names_out()
        print(f"Successfully extracted {len(feature_names)} feature names")
        print(f"Sample feature names: {feature_names[:5]}...")
    except Exception as e:
        print(f"Could not extract feature names: {e}")
        # Fallback: create generic names
        n_features = len(forest_estimator.feature_importances_)
        feature_names = [f'feature_{i}' for i in range(n_features)]
        print(f"Using {len(feature_names)} generic feature names")
    
    # 1. Feature importance
    print("\nGenerating feature importance plot...")
    plt.figure(figsize=(12, 8))
    
    importances = forest_estimator.feature_importances_
    print(f"Feature importances shape: {importances.shape}")
    print(f"Top 5 importance values: {sorted(importances, reverse=True)[:5]}")
    
    indices = np.argsort(importances)[-20:]  # Top 20
    print(f"Plotting top 20 features (indices: {indices[:5]}...{indices[-5:]})")
    
    plt.barh(range(len(indices)), importances[indices], alpha=0.7)
    plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
    plt.xlabel('Feature Importance (Gini Impurity Reduction)')
    plt.title(f'Top 20 Feature Importances - {model_name}')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    importance_path = f"{model_dir}/figures/feature_importance.png"
    plt.savefig(importance_path, dpi=200, bbox_inches='tight')
    print(f"Saved feature importance plot: {importance_path}")
    plt.close()
    
    # 2. ROC Curve
    print("Generating ROC curve plot...")
    plt.figure(figsize=(8, 6))
    mean_fpr = np.linspace(0, 1, 100)
    tprs = []
    
    print(f"Processing {len(model_results['cv_roc_curves'])} CV fold ROC curves...")
    for fold_idx, (fpr, tpr) in enumerate(model_results['cv_roc_curves']):
        tprs.append(np.interp(mean_fpr, fpr, tpr))
        if fold_idx < 3:  # Show details for first 3 folds
            auc_fold = np.trapz(tpr, fpr)
            print(f"  Fold {fold_idx + 1} AUC: {auc_fold:.4f}")
    
    mean_tpr = np.mean(tprs, axis=0)
    std_tpr = np.std(tprs, axis=0)
    mean_auc = np.mean([cv["roc_auc"] for cv in model_results["cv_metrics"]])
    
    print(f"Mean CV AUC: {mean_auc:.4f}")
    print(f"ROC curve std range: {std_tpr.min():.4f} to {std_tpr.max():.4f}")
    
    plt.plot(mean_fpr, mean_tpr, 'b-', 
             label=f'Mean ROC (AUC = {mean_auc:.3f})')
    plt.fill_between(mean_fpr, mean_tpr - std_tpr, mean_tpr + std_tpr, alpha=0.2)
    plt.plot([0, 1], [0, 1], 'k--', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - {model_name}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    roc_path = f"{model_dir}/figures/roc_cv.png"
    plt.savefig(roc_path, dpi=200, bbox_inches='tight')
    print(f"Saved ROC curve plot: {roc_path}")
    plt.close()
    
    print(f"✅ Completed figures for {model_name}")
    print("-" * 40)

print("\n🎨 All figures generated successfully!")
print("Figure types created for each model:")
print("  - Feature importance (top 20 features)")
print("  - ROC curves with confidence intervals")

In [ ]:
# Create summary
print("Creating comprehensive summary of all model results...")
print(f"Processing results for {len(results)} models")
print("=" * 60)

summary_data = []
for model_name, model_results in results.items():
    print(f"\nProcessing summary for: {model_name}")
    
    test_metrics = model_results['test_metrics']
    cv_metrics = model_results['cv_metrics']
    
    # Calculate CV statistics
    cv_aucs = [cv['roc_auc'] for cv in cv_metrics]
    cv_auc_mean = np.mean(cv_aucs)
    cv_auc_std = np.std(cv_aucs)
    
    print(f"  Test AUC: {test_metrics['roc_auc']:.4f}")
    print(f"  CV AUC: {cv_auc_mean:.4f} ± {cv_auc_std:.4f}")
    print(f"  Test F1: {test_metrics['f1']:.4f}")
    print(f"  Best params: {model_results['best_params']}")
    
    summary_data.append({
        'model': model_name,
        'test_auc': test_metrics['roc_auc'],
        'test_ap': test_metrics['average_precision'],
        'test_f1': test_metrics['f1'],
        'test_accuracy': test_metrics['accuracy'],
        'test_precision': test_metrics['precision'],
        'test_recall': test_metrics['recall'],
        'cv_auc_mean': cv_auc_mean,
        'cv_auc_std': cv_auc_std,
        'best_params': str(model_results['best_params']),
        'artifacts_path': model_results['model_dir']
    })

print(f"\nCreating summary DataFrame with {len(summary_data)} models...")
summary_df = pd.DataFrame(summary_data).sort_values('test_auc', ascending=False)

print("Summary DataFrame columns:")
for col in summary_df.columns:
    print(f"  - {col}")

print(f"\nSaving summary to CSV...")
os.makedirs('../outputs/bagging_forests', exist_ok=True)
summary_path = '../outputs/bagging_forests/summary.csv'
summary_df.to_csv(summary_path, index=False)
print(f"Summary saved to: {summary_path}")

print("\n" + "="*70)
print("🏆 BAGGING FORESTS CATEGORY - FINAL RESULTS")
print("="*70)

print(f"\nModel Performance Ranking (by Test AUC):")
print("-" * 50)
for idx, row in summary_df.iterrows():
    rank = summary_df.index.get_loc(idx) + 1
    print(f"{rank:2d}. {row['model']:25s} | AUC: {row['test_auc']:.4f} | AP: {row['test_ap']:.4f} | F1: {row['test_f1']:.4f}")

best_model = summary_df.iloc[0]
print(f"\n🥇 Best Model: {best_model['model']}")
print(f"   Test AUC: {best_model['test_auc']:.4f}")
print(f"   Test AP:  {best_model['test_ap']:.4f}")
print(f"   Test F1:  {best_model['test_f1']:.4f}")
print(f"   CV AUC:   {best_model['cv_auc_mean']:.4f} ± {best_model['cv_auc_std']:.4f}")
print(f"   Best params: {best_model['best_params']}")

print(f"\n📁 All artifacts saved in: ../outputs/bagging_forests/")
print("   - Individual model results in subdirectories")
print("   - Consolidated summary in summary.csv")
print("   - Feature importance and ROC plots for each model")

print(f"\n✨ Analysis complete! Processed {len(results)} forest models successfully.")